In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
# plotting method

def plot_tdf_(df_, tdf_, side_: int = 0, liq_thr=0.1):
    plt.clf()
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type.str.endswith('entry')]
        lc = longs[longs.type == 'close']
        ls = longs[longs.type.str.startswith('stop_loss')]
        ls.price.plot(style='gx')
        le.price.plot(style='b.')
        longs.pos_price.plot(style='b--')
        if 'close_price' in longs.columns:
            longs.close_price.plot(style='r--')
        lc.price.plot(style='r.')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type.str.endswith('entry')]
        sc = shrts[shrts.type == 'close']
        ss = shrts[shrts.type.str.startswith('stop_loss')]
        ss.price.plot(style='gx')
        se.price.plot(style='r.')
        shrts.pos_price.plot(style='r--')
        if 'close_price' in shrts.columns:
            shrts.close_price.plot(style='b--')
        sc.price.plot(style='b.')
    if 'liq_price' in tdf_.columns:
        tdf_.liq_price.where((tdf_.price - tdf_.liq_price).abs() / tdf_.price < liq_thr, np.nan).plot(style='k--')
    return plt


In [ ]:
backtest_config_name = 'lit_nostops'
backtest_config = await prep_backtest_config(backtest_config_name)
session_dirpath = backtest_config['session_dirpath']
session_dirpath

In [ ]:
backtest_config

In [ ]:
ticks = await load_ticks(backtest_config)
df = pd.DataFrame({'price': ticks[:,0], 'buyer_maker': ticks[:,1], 'timestamp': ticks[:,2]})

In [ ]:
results = pd.DataFrame(load_results(session_dirpath + 'results.txt')).T.set_index('index').sort_values('gain', ascending=False)
print('n completed iterations', len(results))
results.drop([k for k in backtest_config['ranges']], axis=1).head(40)
#results.head(40)

In [ ]:
key = results.key.iloc[0]
#key = '306f245f93a9d3f264f4e24c81c77a7332308783d3ea397aa1f7a05822b07f31'
print(key)

In [ ]:
result = results.loc[results.key == key].iloc[0]
backtest_config.update(result)
result.drop('key')

In [ ]:
print(json.dumps(json.load(open(session_dirpath + 'live_config.json')), indent=4))

In [ ]:
tdf = pd.read_csv(f"{session_dirpath}backtest_trades/{key}.csv").set_index('trade_id')
print('price with bid ask entry thresholds')
ema = df.price.ewm(span=result['ema_span'], adjust=False).mean()
bids_ = ema * (1 - result['ema_spread'])
asks_ = ema * (1 + result['ema_spread'])

plt.clf()
df.price.iloc[::100].plot()
bids_.iloc[::100].plot()
asks_.iloc[::100].plot()
plt.savefig(f'{session_dirpath}ema_spread_plot.png')

In [ ]:
def gain_conv(x):
    return x * 100 - 100

lines = []
lines.append(f"net pnl plus fees {result['net_pnl_plus_fees']:.6f}")
lines.append(f"profit sum {result['profit_sum']:.6f}")
lines.append(f"loss sum {result['loss_sum']:.6f}")
lines.append(f"fee sum {result['fee_sum']:.6f}")
lines.append(f"gain {gain_conv(result['gain']):.2f}%")
lines.append(f"n_days {result['n_days']}")
lines.append(f"average_daily_gain percentage {(result['average_daily_gain'] - 1) * 100:.2f}%")
lines.append(f"n trades {result['n_trades']}")
lines.append(f"n closes {result['n_closes']}")
lines.append(f"n reentries {result['n_reentries']}")
lines.append(f"n stop loss closes {result['n_stop_losses']}")
lines.append(f"biggest_pos_size {round(result['biggest_pos_size'], 10)}")
lines.append(f"closest liq {result['closest_liq'] * 100:.4f}%")
lines.append(f"max n hours between consecutive trades {result['max_n_hours_between_consec_trades']:.2f}")
lines.append(f"starting balance {backtest_config['starting_balance']}")
lines.append(f"long: {backtest_config['do_long']}, short: {backtest_config['do_shrt']}")

with open(f"{session_dirpath}backtest_result.txt", 'w') as f:
    for line in lines:
        print(line)
        f.write(line + '\n')


In [ ]:
counter = 0
idxs = []
for row in tdf.itertuples():
    if row.type == 'stop_loss':
        counter += 1
    else:
        if counter > 0:
            idxs.append(row.Index)
        counter = 0
plt.clf()
tdf.pnl_plus_fees_cumsum.plot()
if idxs:
    tdf.net_pnl_plus_fees.loc[idxs].plot(style='ro')
plt.savefig(f'{session_dirpath}pnlcumsum_plot.png')

In [ ]:
# plots are saved in backtesting_results/{exchange}/{symbol}/{session_name}/
n_parts = 7
for z in range(n_parts):
    start_ = z / n_parts
    end_ = (z + 1) / n_parts
    print(start_, end_)
    fig = plot_tdf_(df, tdf.iloc[int(len(tdf) * start_):int(len(tdf) * end_)], liq_thr=0.1)
    fig.savefig(f'{session_dirpath}backtest_{z + 1}of{n_parts}.png')
fig = plot_tdf_(df, tdf, liq_thr=0.1)
fig.savefig(f'{session_dirpath}whole_backtest.png')


In [ ]:
plt.clf()
tdf.pos_size.plot()
plt.savefig(f'{session_dirpath}pos_sizes_plot.png')

In [ ]:
adg_ = tdf.average_daily_gain
print('min max', adg_.min(), adg_.max())
adg_.index = np.linspace(0.0, 1.0, len(tdf))
plt.clf()
adg_.iloc[int(len(tdf) * 0.1):].plot()
plt.savefig(f'{session_dirpath}average_daily_gain_plot.png')

In [ ]:
# visualize behavior
# execute below cell repeatedly (up arrow, shift enter) to see backtest chunk by chunk
# adjust step to set zoom level
step = 120
i = -step

In [ ]:
i += step
tdfc = tdf.iloc[i:i+step]
plot_tdf_(df, tdf.iloc[i:i+step], liq_thr=0.01)

In [ ]:
tdfc.head(60)#.timestamp.diff().values

In [ ]:
tdfc.tail(60)

In [ ]:
closest_liqs = tdf[['closest_long_liq', 'closest_shrt_liq']].min(axis=1).sort_values()
closest_liqs.head()

In [ ]:
i = 0
iloc_ = tdf.index.get_loc(closest_liqs.index[i])
iminus = 10
iplus = 10
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc, liq_thr=0.1)

In [ ]:
tdfc.head(60)

In [ ]:
tsdiffs = tdf.timestamp.diff().sort_values().head()
tsdiffs

In [ ]:
iloc_ = tdf.index.get_loc(tsdiffs.index[0])
tdfz = tdf.iloc[iloc_-5:iloc_+5]
plot_tdf_(df, tdfz)

In [ ]:
tdfz#.timestamp.diff()

In [ ]:
tdf[tdf.type == 'reentry']